# Setting

In [41]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.tool import meta as ml_meta
from Clust.clust.ML.common import ML_api
influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


# 1. set param from Front End

In [42]:
# regression
param1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_trainX',
        "feature_list":['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_trainy',
        "feature_list":["value"]
    },
    'data_y_flag' : 'true',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'split_mode' : 'window_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
        #step_split일 경우만 past_step과 future_step이 존재
        'data_clean_option' : "false"
    },
    
    "model_info" :{
        'model_purpose' : 'regression',
        'model_method' : 'LSTM_rg',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True"}
    }
}

# forecasting
param2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX',
        "feature_list":['Temperature', 'out_temp','sin_hour']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainy',
        "feature_list":['Temperature']
    },
    'data_y_flag' : 'false',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'data_clean_option' : "false",
        'split_mode' : 'step_split', # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
        'past_step':24, #step_split일 경우만 past_step과 future_step이 존재
        'future_step':2
    },
    
    "model_info" :{
        'model_purpose' : 'regression',
        'model_method' : 'LSTM_rg',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True"}
    }
}

# classification
param3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_trainX',
        "feature_list":['col_0', 'col_1','col_2','col_3','col_4','col_5','col_6','col_7','col_8']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_trainy',
        "feature_list":['value']
    },
    'data_y_flag' : 'true',
    'scaler_param':{
        'scaler_flag':'scale', #scale_param,
        'scale_method' :'minmax',
        'scaler_path' :'./scaler/'
    },
    "transform_param":{
        'data_clean_option' : "false",
        'split_mode' : 'window_split' # 현재 data_y_flag=Ture --> 모두 window_split # data_y = False --> step_split
    },
    "model_info" :{
        'model_purpose' : 'classification',
        'model_method' : 'LSTM_cf',
        'model_name' : "None",
        'model_tags' : 'tagstest',
        'train_parameter' : {"lr":0.0001,"weight_decay":0.000001,"n_epochs":5,"batch_size":16},
        'model_parameter' : {"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":"True","num_classes":6, 'rnn_type':'lstm'}
    }
}

# 2. Backend

In [43]:
# 1. Backend Prameter Setting
from Clust.clust.ML.common import ML_api

# parameter tunning
param = param1
param = ML_api.convert_param_for_backend(param)

In [44]:
# 2. train data meta check
train_data_info = ML_api.get_train_data_meta (mongodb_client, param['ingestion_param_X'])
param['train_data_info'] = train_data_info

In [45]:
# 3. Data Preparation
#train_X_array, train_y_array, val_X_array, val_y_array = ML_api.ML_data_preparation(param, influxdb_client)

from Clust.clust.ML.common import ML_pipeline
# 1. Oirignla data ingestion
data_X, data_y = ML_pipeline.Xy_data_preparation(param['ingestion_param_X'], 
                                             param['data_y_flag'], 
                                             param['ingestion_param_y'],
                                             'ms_all', 
                                             influxdb_client)
# 2. Scaling
dataX_scaled, datay_scaled = ML_pipeline.Xy_data_scaling_train(param['ingestion_param_X']['ms_name'], 
                                                                                 data_X, 
                                                                                 param['ingestion_param_y']['ms_name'], 
                                                                                 data_y, 
                                                                                 param['scaler_param'])



# 3.clean column
dataX_scaled = ML_pipeline.clean_low_quality_column(dataX_scaled, param['transform_param'])

param['transform_param']

{'bucket_name': 'integration', 'ms_name': 'regression_energy_cleanLevel4_trainX', 'feature_list': ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']}
None


{'split_mode': 'window_split',
 'data_clean_option': False,
 'nan_process_info': {'type': 'num',
  'ConsecutiveNanLimit': 10000,
  'totalNaNLimit': 100000},
 'max_nan_limit_ratio': 0.5}

In [46]:
# 4. split train/Val
split_ratio = 0.8
train_X, val_X, train_y, val_y, param['transform_param']= ML_pipeline.split_data_by_mode(dataX_scaled, 
                                                                                         datay_scaled, 
                                                                                         split_ratio, 
                                                                                         param['transform_param'])

{'split_mode': 'window_split', 'data_clean_option': False, 'nan_process_info': {'type': 'num', 'ConsecutiveNanLimit': 10000, 'totalNaNLimit': 100000}, 'max_nan_limit_ratio': 0.5, 'future_step': None, 'past_step': 144}
window_split
None


In [47]:
train_X

,Press_mm_hg,RH_1,RH_2,RH_3,RH_4,RH_5,RH_6,RH_7,RH_8,RH_9,...,T4,T5,T6,T7,T8,T9,T_out,Tdewpoint,Visibility,Windspeed
time,,,,,,,,,,,,,,,,,,,,,
2016-01-11 00:00:00+00:00,758.000000,31.666667,31.566667,34.230000,31.000000,45.163333,25.456667,29.000000,34.590000,39.863333,...,19.730000,17.390000,6.090000,22.000000,22.533333,18.500000,5.500000,-4.200000,40.0,7.000000
2016-01-11 00:10:00+00:00,758.083333,31.926667,31.700000,34.290000,31.066667,45.030000,25.963333,28.926667,34.590000,39.900000,...,19.730000,17.390000,5.966667,21.926667,22.463333,18.500000,5.400000,-4.166667,40.0,6.833333
2016-01-11 00:20:00+00:00,758.166667,32.060000,31.956667,34.400000,31.230000,45.060000,25.696667,28.963333,34.663333,39.900000,...,19.700000,17.463333,5.833333,21.856667,22.390000,18.500000,5.300000,-4.133333,40.0,6.666667
2016-01-11 00:30:00+00:00,758.250000,32.260000,32.223333,34.400000,31.356667,45.000000,25.500000,28.963333,34.790000,39.863333,...,19.700000,17.390000,5.763333,21.790000,22.290000,18.463333,5.200000,-4.100000,40.0,6.500000
2016-01-11 00:40:00+00:00,758.333333,32.333333,32.450000,34.400000,31.500000,44.900000,25.693333,28.926667,34.730000,39.863333,...,19.666667,17.390000,5.690000,21.700000,22.230000,18.463333,5.100000,-4.066667,40.0,6.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-03-26 23:10:00+00:00,760.550000,34.333333,31.680000,34.200000,32.030000,37.900000,1.000000,28.637500,34.790000,39.038571,...,23.133333,22.700000,17.390000,23.790000,25.100000,22.890000,13.866667,4.000000,40.0,2.000000
2016-03-26 23:20:00+00:00,760.500000,33.860000,31.412000,34.200000,31.823333,37.595714,1.000000,28.557143,34.700000,39.000000,...,23.066667,22.700000,17.390000,23.777143,25.071429,22.890000,13.933333,3.700000,40.0,2.000000
2016-03-26 23:30:00+00:00,760.450000,33.760000,31.754286,34.200000,32.023333,37.254000,1.000000,28.580000,34.700000,38.928571,...,23.066667,22.700000,17.500000,23.736000,25.000000,22.890000,14.000000,3.400000,40.0,2.000000


In [48]:
print(len(val_X)/128)
print(len(val_y))

21.375
19


In [49]:
# 5. Transform array style
train_X_array, train_y_array = ML_pipeline.transform_data_by_split_mode(param['transform_param'], 
                                                                        train_X, 
                                                                        train_y)
val_X_array, val_y_array = ML_pipeline.transform_data_by_split_mode(param['transform_param'], 
                                                                    val_X, 
                                                                    val_y)

window_size: 144 nan_limit_num: 72
(10944, 24) (72, 144, 24)
(76, 1) (72, 1)
window_size: 144 nan_limit_num: 72
(2736, 24) (19, 144, 24)
(19, 1) (19, 1)


In [ ]:
# inference 테스트 용으로 데이터 저장
# import numpy as np
# np.save('infer_array.npy', val_X_array)

In [50]:
# 4.Training
param = ML_api.ML_training(train_X_array,  train_y_array, val_X_array, val_y_array, param)

{'hidden_size': 64, 'num_layers': 2, 'output_dim': 1, 'dropout': 0.1, 'bidirectional': True}
./Models/LSTM_rg/regression_energy_cleanLevel4_trainX_regression_LSTM_rg_/regression_energy_cleanLevel4_trainX/model.pkl
Start training model
[1/5] Training loss: 229.4901	 Validation loss: 192.8405
[2/5] Training loss: 225.6766	 Validation loss: 191.8355
[3/5] Training loss: 219.1636	 Validation loss: 190.8377
[4/5] Training loss: 229.9409	 Validation loss: 189.8015
[5/5] Training loss: 227.9405	 Validation loss: 188.8138

Training complete in 0m 2s


In [51]:
param

{'ingestion_param_X': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainX',
  'feature_list': ['Press_mm_hg',
   'RH_1',
   'RH_2',
   'RH_3',
   'RH_4',
   'RH_5',
   'RH_6',
   'RH_7',
   'RH_8',
   'RH_9',
   'RH_out',
   'T1',
   'T2',
   'T3',
   'T4',
   'T5',
   'T6',
   'T7',
   'T8',
   'T9',
   'T_out',
   'Tdewpoint',
   'Visibility',
   'Windspeed']},
 'ingestion_param_y': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainy',
  'feature_list': ['value']},
 'data_y_flag': True,
 'scaler_param': {'scaler_flag': 'noscale',
  'scale_method': 'minmax',
  'scaler_path': './scaler/',
  'scaler_file_path': {'XScalerFile': {'fileName': 'scaler.pkl',
    'filePath': None},
   'yScalerFile': {'fileName': 'scaler.pkl', 'filePath': None}}},
 'transform_param': {'split_mode': 'window_split',
  'data_clean_option': False,
  'nan_process_info': {'type': 'num',
   'ConsecutiveNanLimit': 10000,
   'totalNaNLimit': 100000},
  'max_na

In [12]:
# save meta
from Clust.clust.ML.tool import meta
meta_file_name = "./meta.json"
#ml_meta.save_model_meta_into_mongodb(mongodb_client, param, 'model','meta')
meta.save_model_meta_into_local(meta_file_name, param)

======== OK ========


200